# 方程式が成立するかどうかを表す Bool 変数

$x$ を 0-1 決定変数, $a, b$ を整数決定変数として

$$
x = 1 \iff a = b
$$

を実現する制約条件を考えたい. 
まずは不等式の場合から考える. 

## 不等式の場合

$x$ を 0-1 決定変数, $a, b$ を整数決定変数として

$$
x = 1 \iff a \leq b
$$

を実現する制約は big-M 法を使えば次のように線形に書ける. 

\begin{align*}
a &\leq b + M (1 - x) \\
a - 1 &\geq b - M x
\end{align*}

In [1]:
from ortools.sat.python import cp_model

In [2]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f"{v}={self.value(v)}", end=" ")
        print()

    @property
    def solution_count(self):
        return self.__solution_count

In [3]:
lb_a, ub_a = 1, 5
lb_b, ub_b = 1, 5

model = cp_model.CpModel()

x = model.new_bool_var("(a<=b)")
a = model.new_int_var(lb_a, ub_a, "a")
b = model.new_int_var(lb_b, ub_b, "b")
m = ub_b - lb_a + 1

model.add(a <= b + m * (1 - x))
model.add(a - 1 >= b - m * x)

model

In [4]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, a, b])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

(a<=b)=0 a=2 b=1 
(a<=b)=0 a=3 b=1 
(a<=b)=0 a=4 b=1 
(a<=b)=0 a=4 b=2 
(a<=b)=0 a=3 b=2 
(a<=b)=0 a=5 b=2 
(a<=b)=0 a=5 b=1 
(a<=b)=0 a=5 b=3 
(a<=b)=0 a=4 b=3 
(a<=b)=0 a=5 b=4 
(a<=b)=1 a=5 b=5 
(a<=b)=1 a=4 b=4 
(a<=b)=1 a=3 b=4 
(a<=b)=1 a=3 b=3 
(a<=b)=1 a=3 b=5 
(a<=b)=1 a=4 b=5 
(a<=b)=1 a=1 b=5 
(a<=b)=1 a=1 b=4 
(a<=b)=1 a=1 b=3 
(a<=b)=1 a=1 b=2 
(a<=b)=1 a=2 b=2 
(a<=b)=1 a=2 b=3 
(a<=b)=1 a=2 b=4 
(a<=b)=1 a=2 b=5 
(a<=b)=1 a=1 b=1 


In [5]:
print(f"Number of solutions found: {solution_printer.solution_count}\n")

statuses = {
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"status = {statuses[status]}")
print(f"time = {solver.wall_time}")
print(f"objective value = {solver.objective_value}")

Number of solutions found: 25

status = OPTIMAL
time = 0.004654
objective value = 0.0


## 方程式の場合

big-M 法を用いれば方程式の場合も線形に表すことができる. 

- $x = 1 \iff a \leq b$ を表す制約
  - $a \leq b + M (1 - x)$
  - $a - 1 \geq b - M x$
- $y = 1 \iff a \geq b$ を表す制約
  - $b \leq a + M (1 - y)$
  - $b - 1 \geq a - M y$
- $z = x \land y$ を表す制約
  - $z + 1 \geq x + y$
  - $2 z \leq x + y$

これで
$z = 1 \iff a = b$
となる. 

In [6]:
lb_a, ub_a = 1, 5
lb_b, ub_b = 1, 5

model = cp_model.CpModel()

x = model.new_bool_var("(a<=b)")
y = model.new_bool_var("(a>=b)")
z = model.new_bool_var("(a==b)")
a = model.new_int_var(lb_a, ub_a, "a")
b = model.new_int_var(lb_b, ub_b, "b")
m = max(ub_a - lb_b + 1, ub_b - lb_a + 1)

model.add(a <= b + m * (1 - x))
model.add(a - 1 >= b - m * x)
model.add(b <= a + m * (1 - y))
model.add(b - 1 >= a - m * y)
model.add(z + 1 >= x + y)
model.add(2 * z <= x + y)

model

In [7]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([z, x, y, a, b])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

(a==b)=0 (a<=b)=1 (a>=b)=0 a=1 b=2 
(a==b)=1 (a<=b)=1 (a>=b)=1 a=1 b=1 
(a==b)=1 (a<=b)=1 (a>=b)=1 a=2 b=2 
(a==b)=1 (a<=b)=1 (a>=b)=1 a=3 b=3 
(a==b)=0 (a<=b)=1 (a>=b)=0 a=2 b=3 
(a==b)=0 (a<=b)=1 (a>=b)=0 a=1 b=3 
(a==b)=0 (a<=b)=1 (a>=b)=0 a=1 b=4 
(a==b)=0 (a<=b)=1 (a>=b)=0 a=1 b=5 
(a==b)=0 (a<=b)=1 (a>=b)=0 a=2 b=5 
(a==b)=0 (a<=b)=1 (a>=b)=0 a=2 b=4 
(a==b)=0 (a<=b)=1 (a>=b)=0 a=3 b=4 
(a==b)=0 (a<=b)=1 (a>=b)=0 a=3 b=5 
(a==b)=0 (a<=b)=1 (a>=b)=0 a=4 b=5 
(a==b)=1 (a<=b)=1 (a>=b)=1 a=5 b=5 
(a==b)=1 (a<=b)=1 (a>=b)=1 a=4 b=4 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=5 b=4 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=4 b=3 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=5 b=3 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=5 b=1 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=5 b=2 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=4 b=2 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=4 b=1 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=3 b=1 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=3 b=2 
(a==b)=0 (a<=b)=0 (a>=b)=1 a=2 b=1 


In [8]:
print(f"Number of solutions found: {solution_printer.solution_count}\n")

statuses = {
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"status = {statuses[status]}")
print(f"time = {solver.wall_time}")
print(f"objective value = {solver.objective_value}")

Number of solutions found: 25

status = OPTIMAL
time = 0.002258
objective value = 0.0


## `only_enforce_if()` の利用

後から気づいたが Google OR-Tools には `only_enforce_if()` 関数があり, 
特定の Bool 変数が `True` のときのみ制約を ON にすることができる. 

### 同値でなくてもよい場合

下記を直接制約に加える. 

$$
x = 1 \Longrightarrow a = b
$$

In [9]:
lb_a, ub_a = 1, 5
lb_b, ub_b = 1, 5

model = cp_model.CpModel()

x = model.new_bool_var("(a==b)")
a = model.new_int_var(lb_a, ub_a, "a")
b = model.new_int_var(lb_b, ub_b, "b")

model.add(a == b).only_enforce_if(x)

model

In [10]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, a, b])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

(a==b)=0 a=1 b=1 
(a==b)=0 a=2 b=1 
(a==b)=0 a=2 b=2 
(a==b)=0 a=1 b=2 
(a==b)=0 a=3 b=2 
(a==b)=0 a=3 b=1 
(a==b)=0 a=3 b=3 
(a==b)=0 a=2 b=3 
(a==b)=0 a=1 b=3 
(a==b)=0 a=4 b=3 
(a==b)=0 a=4 b=2 
(a==b)=0 a=4 b=1 
(a==b)=0 a=4 b=4 
(a==b)=0 a=3 b=4 
(a==b)=0 a=2 b=4 
(a==b)=0 a=1 b=4 
(a==b)=0 a=5 b=4 
(a==b)=0 a=5 b=3 
(a==b)=0 a=5 b=2 
(a==b)=0 a=5 b=1 
(a==b)=0 a=5 b=5 
(a==b)=0 a=4 b=5 
(a==b)=0 a=3 b=5 
(a==b)=0 a=2 b=5 
(a==b)=0 a=1 b=5 
(a==b)=1 a=1 b=1 
(a==b)=1 a=2 b=2 
(a==b)=1 a=3 b=3 
(a==b)=1 a=4 b=4 
(a==b)=1 a=5 b=5 


In [11]:
print(f"Number of solutions found: {solution_printer.solution_count}\n")

statuses = {
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"status = {statuses[status]}")
print(f"time = {solver.wall_time}")
print(f"objective value = {solver.objective_value}")

Number of solutions found: 30

status = OPTIMAL
time = 0.001708
objective value = 0.0


### 同値にしたい場合

上記の

$$
x = 1 \Longrightarrow a = b
$$

に加えてその裏を制約に入れることで同値にできる: 

$$
x = 0 \Longrightarrow a \ne b
$$

In [12]:
lb_a, ub_a = 1, 5
lb_b, ub_b = 1, 5

model = cp_model.CpModel()

x = model.new_bool_var("(a==b)")
a = model.new_int_var(lb_a, ub_a, "a")
b = model.new_int_var(lb_b, ub_b, "b")

model.add(a == b).only_enforce_if(x)
model.add(a != b).only_enforce_if(x.negated())

model

In [13]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, a, b])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

(a==b)=0 a=1 b=2 
(a==b)=0 a=2 b=1 
(a==b)=0 a=3 b=1 
(a==b)=0 a=4 b=1 
(a==b)=0 a=4 b=2 
(a==b)=0 a=3 b=2 
(a==b)=0 a=5 b=2 
(a==b)=0 a=5 b=1 
(a==b)=0 a=5 b=3 
(a==b)=0 a=4 b=3 
(a==b)=0 a=5 b=4 
(a==b)=0 a=4 b=5 
(a==b)=0 a=3 b=4 
(a==b)=0 a=3 b=5 
(a==b)=0 a=2 b=5 
(a==b)=0 a=2 b=4 
(a==b)=0 a=2 b=3 
(a==b)=0 a=1 b=3 
(a==b)=0 a=1 b=4 
(a==b)=0 a=1 b=5 
(a==b)=1 a=1 b=1 
(a==b)=1 a=2 b=2 
(a==b)=1 a=3 b=3 
(a==b)=1 a=4 b=4 
(a==b)=1 a=5 b=5 


In [14]:
print(f"Number of solutions found: {solution_printer.solution_count}\n")

statuses = {
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"status = {statuses[status]}")
print(f"time = {solver.wall_time}")
print(f"objective value = {solver.objective_value}")

Number of solutions found: 25

status = OPTIMAL
time = 0.001345
objective value = 0.0


## 応用: 誰が祠を

X のポスト([https://x.com/mrsolyu/status/1846512850879275074](https://x.com/mrsolyu/status/1846512850879275074))でこういった問題があったので定式化して犯人を求める. 

> お前達の誰かが、あの祠を壊したんか！？
> 
> A「俺がやりました」\
> B「犯人は2人いる」\
> C「Dが犯人でないなら僕が犯人」\
> D「4人の中で嘘つきは奇数人」
> 
> 犯人はこの中にいるはずじゃ。そして呪いで嘘しかつけなくなっておるわい。
> 誰が祠を壊したかのう？

総当たりで探索しても一瞬で終わる程度の規模ではあるが, 練習のために定式化と実装を行う. 

### 定式化

#### 変数

- $x_A, x_B, x_C, x_D$: A ~ D が嘘つきのとき $1$, 正直もののとき $0$
- $y_A, y_B, y_C, y_D$: A ~ D が犯人のとき $1$, そうでないとき $0$

#### 制約

- 祠を壊したものは呪いで嘘しかつけなくなっている
  - $y_* <= x_*$
- A「俺がやりました」
  - $x_A = 0 \Longrightarrow y_A = 1$
  - $x_A = 1 \Longrightarrow y_A = 0$
  - 上記 2 つをまとめて $x_A = 1 - y_A$ と書ける
- B「犯人は2人いる」
    - $x_B = 0 \Longrightarrow y_A + y_B + y_C + y_D = 2$
    - $x_B = 1 \Longrightarrow y_A + y_B + y_C + y_D \ne 2$
- C「Dが犯人でないなら僕が犯人」
  - $x_C = 0 \Longrightarrow$ 「$y_D = 0 \Longrightarrow y_C = 1$」だがこれは $1 - x_C \leq y_C + y_D$ と同値
  - $x_C = 1 \Longrightarrow$ 「$y_D = 0 \land y_C = 0$」だがこれは $2 (1 - x_C) \geq y_C + y_D$ と同値
- D「4人の中で嘘つきは奇数人」
  - $x_D = 0 \Longrightarrow x_A + x_B + x_C + x_D \equiv 1 \mod 2$
  - $x_D = 1 \Longrightarrow x_A + x_B + x_C + x_D \equiv 0 \mod 2$
  - 上記をまとめて $x_A + x_B + x_C + x_D \equiv 1 - x_D \mod 2$ として実装する
  - この条件は線形にすることができる.
    $e, o$ を 0-1 決定変数とし, 嘘つきの数が偶数人か奇数人かに対応させるとする.
    この条件は次のように書ける. $s_e, z$ を整数決定変数として,

    - $e + o = 1$
    - $x_D = e$
    - $s_e = 2 * z$
    - $x_A + x_B + x_C + x_D = s_e + o$

    とすればよい.
    こうして A から D までの全ての条件は線形制約で記述できる. 

### 実装

In [15]:
model = cp_model.CpModel()

suspects = ["A", "B", "C", "D"]

liar = {s: model.new_bool_var(f"{s}_is_liar") for s in suspects}
culprit = {s: model.new_bool_var(f"{s}_is_culprit") for s in suspects}

# 祠を壊したものは呪いで嘘しかつけなくなっている
for s in suspects:
    model.add_implication(culprit[s], liar[s])

# A「俺がやりました」
model.add_bool_xor(liar["A"], culprit["A"])

# B「犯人は2人いる」
model.add(sum(culprit[s] for s in suspects) == 2).only_enforce_if(liar["B"].negated())
model.add(sum(culprit[s] for s in suspects) != 2).only_enforce_if(liar["B"])

# C「Dが犯人でないなら僕が犯人」
model.add_implication(culprit["D"].negated(), culprit["C"]).only_enforce_if(liar["C"].negated())
model.add(culprit["C"] == 0).only_enforce_if(liar["C"])
model.add(culprit["D"] == 0).only_enforce_if(liar["C"])

# D「4人の中で嘘つきは奇数人」
n_liar = model.new_int_var(0, len(suspects), "num_of_liars")
model.add(n_liar == sum(liar[s] for s in suspects))
model.add_modulo_equality(liar["D"].negated(), n_liar, 2)

# 追加: 犯人は必ず 1 人はいる
model.add(sum(culprit[s] for s in suspects) >= 1)

In [16]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(list(liar.values()) + list(culprit.values()))
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

A_is_liar=1 B_is_liar=1 C_is_liar=1 D_is_liar=0 A_is_culprit=0 B_is_culprit=1 C_is_culprit=0 D_is_culprit=0 
A_is_liar=1 B_is_liar=1 C_is_liar=1 D_is_liar=1 A_is_culprit=0 B_is_culprit=1 C_is_culprit=0 D_is_culprit=0 


In [17]:
print(f"Number of solutions found: {solution_printer.solution_count}\n")

statuses = {
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"status = {statuses[status]}")
print(f"time = {solver.wall_time}")
print(f"objective value = {solver.objective_value}")

Number of solutions found: 2

status = OPTIMAL
time = 0.0007970000000000001
objective value = 0.0


### 実装(線形版)

In [18]:
# 祠を壊したものは呪いで嘘しかつけなくなっている
for s in suspects:
    model.add(culprit[s] <= liar[s])

# A「俺がやりました」
model.add(1 - liar["A"] == culprit["A"])

# B「犯人は2人いる」
y = model.new_bool_var("(culprits<=2)")
z = model.new_bool_var("(culprits>=2)")
m = 10
model.add(sum(culprit[s] for s in suspects) <= 2 + m * (1 - y))
model.add(sum(culprit[s] for s in suspects) - 1 >= 2 - m * y)
model.add(2 <= sum(culprit[s] for s in suspects) + m * (1 - z))
model.add(2 - 1 >= sum(culprit[s] for s in suspects) - m * z)
model.add((1 - liar["B"]) + 1 >= y + z)
model.add(2 * (1 - liar["B"]) <= y + z)

# C「Dが犯人でないなら僕が犯人」
model.add(1 - liar["C"] <= culprit["C"] + culprit["D"])
model.add(2 * (1 - liar["C"]) >= culprit["C"] + culprit["D"])

# D「4人の中で嘘つきは奇数人」
e = model.new_bool_var("n_liar_is_even")
o = model.new_bool_var("n_liar_is_odd")
model.add(e + o == 1)
model.add(liar["D"] == e)
se = model.new_int_var(0, len(liar) // 2, "n_liar//2")
model.add(sum(liar[s] for s in suspects) == 2 * se + o)

# 追加: 犯人は必ず 1 人はいる
model.add(sum(culprit[s] for s in suspects) >= 1)

model

In [19]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(list(liar.values()) + list(culprit.values()))
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

A_is_liar=1 B_is_liar=1 C_is_liar=1 D_is_liar=0 A_is_culprit=0 B_is_culprit=1 C_is_culprit=0 D_is_culprit=0 
A_is_liar=1 B_is_liar=1 C_is_liar=1 D_is_liar=1 A_is_culprit=0 B_is_culprit=1 C_is_culprit=0 D_is_culprit=0 


In [20]:
print(f"Number of solutions found: {solution_printer.solution_count}\n")

statuses = {
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"status = {statuses[status]}")
print(f"time = {solver.wall_time}")
print(f"objective value = {solver.objective_value}")

Number of solutions found: 2

status = OPTIMAL
time = 0.0006940000000000001
objective value = 0.0


### 結果

犯人が 1 人以上いると仮定すると犯人は B でそれ以外は無実という結果になった. 
全員が犯人でないケースもあり得たが今回は除外した. 
嘘つきかどうかに関しては D 以外は全員嘘つきで確定していて, D は嘘つきでも正直ものでもどちらでも整合した. 

`add_modulo_equality()` の引数に式をそのまま入れてしまうと `MODEL_INVALID` になってしまった. 
他の関数, 例えば `add_multiplication_equality()` でも同様のことが起こったため, 
線形でない制約を追加する際は式を新しい変数に格納してから渡すと安全そう. 

また, `add_modulo_equality()` の返り値に `only_enforce_if()` を繋げたら `MODEL_INVALID` となってしまった. 
ドキュメントには書かれていなかったが `only_enforce_if()` が使える制約と使えない制約があるようで, 
例えば `add_bool_xor()` は明確に

> In contrast to add_bool_or and add_bool_and, it does not support .only_enforce_if().

と書かれている. 
(`model.add(a != b)` には `only_enforce_if()` をつなげることができたのでなぜこうなっているかは謎)